In [1]:
import os

os.getcwd()

'C:\\Users\\Ben04\\Documents\\Coding\\PowerOf10Analysis'

In [2]:
import pickle
import plotly.graph_objects as go
import pprint
import numpy as np
import pandas as pd

# Invert the process: Load the pickled data
with open('athlete_data.pkl', 'rb') as f:
    (listOfAthletes, listOfDFs, listOfClubs, athleteInfo, clubsDF,
     athleteLines, athleteFigs, clubLines, groupLine) = pickle.load(f)


In [3]:
def clubFilter(athleteList, searchClub):
    dfClub = clubsDF.loc[clubsDF['club'] == searchClub]
    athletes = dfClub['athlete_id'].to_numpy()
    athleteList = np.array(athleteList)
    filtered_athletes = np.intersect1d(athletes, athleteList)

    return filtered_athletes.tolist()

def ageFilter(athleteList, yearOfBirth):
    filtered_athletes = athleteInfo[
        (athleteInfo.index.isin(athleteList)) &
        (athleteInfo['birthyear'] == yearOfBirth)
        ].index
    return filtered_athletes


In [4]:
def clubImprovement(club, label, minAge=0, maxAge=100):
    groupList = clubFilter(listOfAthletes, club)

    totalimprovment = 0
    athletecount = 0
    for athlete in groupList:
        line, minn, maxx, raw = athleteLines[athlete][label]
        if line:
            values = clubsDF.loc[(clubsDF['club'] == club) & (clubsDF['athlete_id'] == athlete), label].values
            print(athleteInfo.loc[athlete]['full_name'])
            print(athlete)
            for value in values:
                if value['end_year'] > minn and value['end_year'] > minAge:
                    starting = max(value['start_year'], minn, minAge)
                    ending = min((value['end_year'] + 1), maxx, maxAge)
                    if starting < ending:
                        startvalue = line(starting)
                        endvalue = line(ending)

                        improvement = ((endvalue - startvalue) / startvalue) * 100
                        print(f'From {startvalue} at {starting} to {endvalue}: {ending}')
                        print(f'Improvement: {improvement}')
                        totalimprovment += improvement
                        athletecount += 1

    if athletecount > 0:
        avgimprovment = totalimprovment / athletecount
        print("Average improvment of group {} is {}".format(label, avgimprovment))
    else:
        print("NO ATHLETES")
        
def groupImprovement(athleteList, label, start, end):
    totalimprovment = 0
    athletecount = 0
    for athlete in athleteList:
        line, minn, maxx, raw = athleteLines[athlete][label]
        if line:
            print(athleteInfo.loc[athlete]['full_name'])
            print(athlete)
            if end > minn and start < maxx:
                starting = max(start, minn)
                ending = min(end, maxx)
                if starting < ending:
                    startvalue = line(starting)
                    endvalue = line(ending)

                    improvement = ((endvalue - startvalue) / startvalue) * 100
                    print(f'From {startvalue} at {starting} to {endvalue}: {ending}')
                    print(f'Improvement: {improvement}')

                    totalimprovment += improvement
                    athletecount += 1

    print(athletecount)
    if athletecount > 0:
        avgimprovment = totalimprovment / athletecount
        print("Average improvment of group {} is {}".format(label, avgimprovment))
        return avgimprovment
    else:
        print("NO ATHLETES")
        return None


In [5]:
line, minn, maxx = clubLines['Birmingham Uni']['age']
improvement = ((line(21) - line(18)) / line(18)) * 100
print(improvement)

6.405859761385664


In [6]:
def highestperformers(athleteList, label, x_input):
    athleteList = np.array(athleteList)
    df = pd.DataFrame(columns=['athlete', 'athlete_name', 'best_performance'])

    athlete_id = 0
    athleteName = ''
    athletePerformance = 0
    for count, athlete in enumerate(athleteList):
        athlete_id = athlete
        athleteName = athleteInfo._get_value(athlete, 'full_name')
        
        start = athleteLines[athlete][label][1]
        end = athleteLines[athlete][label][2]
        rawdata = athleteLines[athlete][label][3]
        if rawdata is not None and athlete_id is not None and athleteName is not None and start <= x_input:
            validdata = rawdata[(rawdata['x'] > x_input) & (rawdata['x'] < x_input + 1)]['y']
            athletePerformance = validdata.max() if not validdata.empty else np.nan
            df.loc[count] = [athlete_id, athleteName, athletePerformance]

    df = df.sort_values(by='best_performance', ascending=False)
    return df

In [7]:
ratings2024 = highestperformers(listOfAthletes, 'dec_date', 2024)
pd.set_option('display.max_rows', None)
print(ratings2024[:100])

       athlete           athlete_name  best_performance
7779    556235             Max Burgin       1174.427299
3069     63342          Jake Wightman       1171.894790
2023     16670              Sam Atkin       1171.532104
4661     81741              Josh Kerr       1171.363514
6181    439836           George Mills       1165.006240
3818     37208           Neil Gourley       1163.080916
5822    110447          Emile Cairess       1162.254922
7697    444384           Ben Pattison       1153.996531
8515    512438        Mahamed Mahamed       1150.332938
5231    113232          Daniel Rowden       1148.496237
3728    155067           Elliot Giles       1135.718069
4478     81765          Patrick Dever       1135.640617
5767     90724    William Battershill       1127.299472
7870    750931              Adam Fogg       1124.359420
6162     84701        Thomas Randolph       1123.689378
6928    227724         Alex Botterill       1123.620927
13192   634495    Alex Haydock-Wilson       1118

In [8]:
ages = np.arange(14,30)
agerankings = {}
for age in ages:
    agerankings[age] = highestperformers(listOfAthletes, 'age', age)
    
years = np.arange(2000,2025)
yearrankings = {}
for year in years:
    yearrankings[year] = highestperformers(listOfAthletes, 'dec_date', year)


In [106]:
df = agerankings[23]

pprint.pprint(df[:200])
pprint.pprint(df.index.get_loc(df[df['athlete_name'] == 'Noah Bennett'].index[0]) + 1)

       athlete           athlete_name  best_performance
7697    444384           Ben Pattison       1160.644852
5789    186398           Jake Heyward       1150.643111
5231    113232          Daniel Rowden       1140.368312
6744      6004          Martyn Rooney       1138.009656
12059  1036061          Charles Hicks       1125.058633
4661     81741              Josh Kerr       1120.510312
7056    538179            Thomas Keen       1117.628288
251       1388            Curtis Robb       1116.852983
6181    439836           George Mills       1115.923805
6928    227724         Alex Botterill       1110.046568
4399    377196          Kyle Langford       1108.910221
3069     63342          Jake Wightman       1106.758053
6569     80805         Piers Copeland       1104.559740
233        909         Tom Lancashire       1103.903770
2452     18026  Charlie Da'Vall Grice       1103.582281
217       1068          James McIlroy       1102.973084
3647     21594           Nigel Levine       1100

In [14]:
import plotly.express as px

athletes = {
    'Josh Kerr': [],
    'Jake Wightman': [],
    'Max Burgin': [],
    'Mohamed Farah': [],
    'Sam Atkin': []
}

years = np.arange(2010, 2025)

for athlete in athletes:
    for year in years:
        df = yearrankings[year]
        # Check if the athlete is in the dataframe
        athlete_data = df[df['athlete_name'] == athlete]
        
        if not athlete_data.empty:
            # Extract the 'best_performance' value
            best_performance = athlete_data['best_performance'].values[0]
            
            if not np.isnan(best_performance):
                # Append the ranking (1-based)
                athletes[athlete].append(df.index.get_loc(athlete_data.index[0]) + 1)
            else:
                # Append NaN if performance is NaN
                athletes[athlete].append(np.nan)
        else:
            # Append NaN if the athlete is not in the dataframe
            athletes[athlete].append(np.nan)

# Create the plot
fig = go.Figure()
colors = px.colors.qualitative.Plotly


for counter, athlete in enumerate(athletes):
    fig.add_trace(go.Scatter(x=years, y=athletes[athlete], name=athlete,line=dict(color=colors[counter])))

fig.update_yaxes(range=[50,0])
fig.update_xaxes(title='Year')
fig.update_yaxes(title='UK Ranking')
fig.updatelayout(template='plotly_dark')
fig.show()


AttributeError: 'Figure' object has no attribute 'updatelayout'

In [132]:
import plotly.express as px

data = []
for year in years:
    data.append({
        'year': year,
        '1st': yearrankings[year].iloc[0]['best_performance'],
        '10th': yearrankings[year].iloc[10]['best_performance'] if len(yearrankings[year]) > 10 else None,
        '100th': yearrankings[year].iloc[99]['best_performance'] if len(yearrankings[year]) > 99 else None
    })

values = pd.DataFrame(data)

colors = px.colors.qualitative.Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=values['year'], y = values['1st'], mode='lines', line=dict(color=colors[0]),  name='1st'))
fig.add_trace(go.Scatter(x=values['year'], y = values['10th'], mode='lines', line=dict(color=colors[1]), name='10th'))
fig.add_trace(go.Scatter(x=values['year'], y = values['100th'], mode='lines', line=dict(color=colors[2]), name='100th'))
fig.update_yaxes(range=[850,1200], title='World Athletics Points')
fig.update_xaxes(title='Year')
fig.show()

In [14]:
import numpy as np

def clubPerformance(listOfAthletes, club, label, year):
    performances = []
    athletecount = 0
    minAge = year
    maxAge = year + 1
    for athlete in listOfAthletes:
        line, minn, maxx, raw = athleteLines[athlete][label]
        if line:
            values = clubsDF.loc[(clubsDF['club'] == club) & (clubsDF['athlete_id'] == athlete), label].values
            # print(athleteInfo.loc[athlete]['full_name'])
            for value in values:
                if minn < value['end_year'] <= maxAge and value['end_year'] >= minAge and maxAge - value['start_year'] < 5 and minAge - athleteInfo.loc[athlete]['birthyear'] >=18:
                    starting = max(value['start_year'], minn, minAge)
                    ending = min((value['end_year'] + 1), maxx, maxAge)
                    # print(f'starting {starting}')
                    # print(f'ending {ending}')
                    if starting < ending:
                        validp = np.arange(starting,ending, 0.1)
                        values = [line(x) for x in validp]
                        if values is not []:
                            maxperformance = max(values)
                            performances.append(maxperformance)
                            athletecount += 1


    if athletecount == 0:
        return None
    else:
        return performances




In [126]:
data = []

df23 = yearrankings[2023]
df24 = yearrankings[2024]

for athlete in listOfAthletes:
    _23value = df23.loc[df23['athlete']==athlete]['best_performance'].values
    _24value = df24[df24['athlete'] == athlete]['best_performance'].values
    if len(_23value) > 0 and len(_24value) > 0:
        _23value = _23value[0]
        _24value = _24value[0]
        improvement = _24value - _23value
        percent_improvement = improvement / _23value
        
        data.append({
            'athlete': athlete,
            'athlete_name': athleteInfo._get_value(athlete, 'full_name'),
            '2023_performance': _23value,
            '2024_performance': _24value,
            'value_improvement': improvement,
            'percent_improvement': percent_improvement
        })
    
improvementdf = pd.DataFrame(data)
pprint.pprint(improvementdf[:100])
improvementdf = improvementdf.sort_values(by='value_improvement', ascending=False)

    athlete        athlete_name  2023_performance  2024_performance  \
0      4261            Ed Aston      8.965324e+02      7.959943e+02   
1      1447       Chris Saville      4.299080e-09      1.023967e-10   
2      8254           Rick Ward      3.865277e+00      6.725986e-01   
3      1663       James Tydeman      1.100499e+01      2.282876e+00   
4     12316       Andrew Osagie      3.862892e+02      1.650589e+02   
5      4944         James Shane      5.841005e+00      1.047179e+00   
6      4927        Mark Woodley      7.444286e+01      2.187685e+01   
7      1518        Robert Smith      3.094100e+02      3.712851e+02   
8      1102         Colin Miles      3.321311e-04      1.974507e-05   
9       172       Peter Bridger      9.467871e-05      4.904090e-06   
10     3444        Danny Crates      5.582083e+02      3.378373e-01   
11    17347            Tom Love      1.092012e-03      7.197651e-05   
12     3883         Adam Hickey      8.939422e+02      8.337303e+02   
13    

In [18]:
import numpy as np
import plotly.graph_objects as go

# Define clubs and their average performance lists
clubs = {
    "Birmingham Uni": {'data': {}, 'avg': []},
    "Loughborough Students": {'data': {}, 'avg': []},
    "St. Mary's Uni": {'data': {}, 'avg': []},
    "Leeds Uni": {'data': {}, 'avg': []},
    "Cambridge Uni": {'data': {}, 'avg': []}
}

x_input = np.arange(2007, 2025)
fig = go.Figure()

# Iterate through each year and calculate performance for each club
for year in x_input:
    for club_name, club_dict in clubs.items():
        club_data = clubPerformance(clubFilter(listOfAthletes, club_name), club_name, 'dec_date', year)
        club_dict['data'][year] = club_data
        
        if club_data is not None:
            club_dict['avg'].append(np.average(club_data))
        else:
            club_dict['avg'].append(np.nan)  # Append NaN for missing data

# Add traces for each club to the plot
for club_name, club_dict in clubs.items():
    fig.add_trace(go.Scatter(x=x_input + 0.5, y=club_dict['avg'], name=club_name))

# Show the figure
fig.show()
